<h3> Leetcode Hard Problem 3 | Market Analysis 2 | Complex SQL 9 </h3>

In [0]:
%sql
drop table if exists users;
create table users (
user_id         int     ,
 join_date       date    ,
 favorite_brand  varchar(50));

 create table orders (
 order_id       int     ,
 order_date     date    ,
 item_id        int     ,
 buyer_id       int     ,
 seller_id      int 
 );

 create table items
 (
 item_id        int     ,
 item_brand     varchar(50)
 );


 insert into users values (1,'2019-01-01','Lenovo'),(2,'2019-02-09','Samsung'),(3,'2019-01-19','LG'),(4,'2019-05-21','HP');

 insert into items values (1,'Samsung'),(2,'Lenovo'),(3,'LG'),(4,'HP');

 insert into orders values (1,'2019-08-01',4,1,2),(2,'2019-08-02',2,1,3),(3,'2019-08-03',3,2,3),(4,'2019-08-04',1,4,2)
 ,(5,'2019-08-04',1,3,4),(6,'2019-08-05',2,2,4);

num_affected_rows,num_inserted_rows
6,6


In [0]:
%sql
select * from users

user_id,join_date,favorite_brand
1,2019-01-01,Lenovo
2,2019-02-09,Samsung
3,2019-01-19,LG
4,2019-05-21,HP


In [0]:
%sql
select * from orders

order_id,order_date,item_id,buyer_id,seller_id
1,2019-08-01,4,1,2
2,2019-08-02,2,1,3
3,2019-08-03,3,2,3
4,2019-08-04,1,4,2
5,2019-08-04,1,3,4
6,2019-08-05,2,2,4


In [0]:
%sql 
select * from items

item_id,item_brand
1,Samsung
2,Lenovo
3,LG
4,HP


<h3> Solution in sql(hive) </h3>

In [0]:
%sql
with cte as(
select *,row_number()over(partition by seller_id order by order_date asc)rn from orders)
select user_id as seller_id, case when favorite_brand=item_brand then 'Yes' else 'No' end as 2nd_item_fav_brand  from users
left join cte on cte.seller_id=users.user_id and rn=2
left join items on items.item_id = cte.item_id


seller_id,2nd_item_fav_brand
1,No
2,Yes
3,Yes
4,No


<h3> Solution in pyspark </h3>

In [0]:
from pyspark.sql.functions import col,row_number,asc,when
from pyspark.sql.window import Window

In [0]:
df_u=spark.sql("select * from users")
df_o=spark.sql("select * from orders")
df_i=spark.sql("select * from items")
df_u.show()
df_o.show()
df_i.show()

+-------+----------+--------------+
|user_id| join_date|favorite_brand|
+-------+----------+--------------+
|      1|2019-01-01|        Lenovo|
|      2|2019-02-09|       Samsung|
|      3|2019-01-19|            LG|
|      4|2019-05-21|            HP|
+-------+----------+--------------+

+--------+----------+-------+--------+---------+
|order_id|order_date|item_id|buyer_id|seller_id|
+--------+----------+-------+--------+---------+
|       1|2019-08-01|      4|       1|        2|
|       2|2019-08-02|      2|       1|        3|
|       3|2019-08-03|      3|       2|        3|
|       4|2019-08-04|      1|       4|        2|
|       5|2019-08-04|      1|       3|        4|
|       6|2019-08-05|      2|       2|        4|
+--------+----------+-------+--------+---------+

+-------+----------+
|item_id|item_brand|
+-------+----------+
|      1|   Samsung|
|      2|    Lenovo|
|      3|        LG|
|      4|        HP|
+-------+----------+



In [0]:
df_o=df_o.withColumn("rn",row_number().over(Window.partitionBy("seller_id").orderBy(col("order_date").asc())))
df_rn=df_o.filter(df_o.rn==2).select("seller_id","item_id")
df_u_rn=df_u.join(df_rn,df_rn.seller_id==df_u.user_id,"left").select("user_id","item_id","favorite_brand")
df_f=df_u_rn.join(df_i,"item_id","left")
df_f=df_f.withColumnRenamed("user_id","seller_id")
df_final=df_f.withColumn("2nd_item_fav_brand",when(df_f.favorite_brand==df_f.item_brand,'Yes').otherwise("no"))
df_final.select("seller_id","2nd_item_fav_brand").show()

+---------+------------------+
|seller_id|2nd_item_fav_brand|
+---------+------------------+
|        1|                no|
|        2|               Yes|
|        3|               Yes|
|        4|                no|
+---------+------------------+

